# Ray tracing basics

The goal of this example is to show how to perform a very simple ray-tracing of a generic shape and visualize the results.

In this example we will show how to compute the interaction between the impinging rays and the shape accounting for specular reflection and diffusion.

We will also show how to determine which parts of the shape are impinged by the first, second, third, .., reflectiona and represent it graphically.

Note: Since here we want to show very basic functionalities of the library, we will use directly the core utilities, without using the more complex classes (Spacecraft, PixelPlane, etc.)

In [1]:
import trimesh as tm
import numpy as np
import spiceypy as sp
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import to_rgba_array

from pyRTX.classes.PixelPlane import PixelPlane
from pyRTX.core import utils_rt

We can apply the same example to different shapes. Choose here below which one

In [2]:
#obj = 'Annulus'
obj = 'Sphere'

In [3]:
if obj == 'Annulus':
    mesh = tm.creation.annulus(1,2,height = 1)
elif obj == 'Sphere':
    mesh = mesh = tm.creation.icosphere(radius = 1, subdivisions = 4)
    

Let's set up the origin of the ray-tracing rays. We set up the so-called Pixel Plane at a certain RA/DEC (or lat/lon) with respect to the base reference frame.
The PixelPlane object method `.dump()` returns the origin coordinates and directions of the ray tracing rays

In [4]:
RA = 0
DEC = 0

pixelPlane = PixelPlane(
        spacecraft = 'None',
        mode = 'Fixed',
        distance = 3,
        width = 2,
        height = 2,
        ray_spacing = 0.1,
        lon = RA,
        lat = DEC,
        units = 'km',
)
        

ray_origins, ray_directions = pixelPlane.dump()

We can use these points/directions as the input of the ray tracer.

Here we set the options of the ray tracer:

- number of reflections
- diffusion activated/not activated


`utils_rt.RTXkernel()` returns:
- index_tri_container : an array of arrays containing the indexes of the mesh vertices that have been hit by the tracing rays. if more than one bounce is computed then, the array will be structured as follows [ [indexes_bounce_1], [indexes_bounce_2],...]
- index_ray_container : an array of arrays containing the indexes of the rays that have hit the mesh. 
- locations_container : an array of arrays containing the coordinates of the hit points
- ray_directions_container :  an array containing the direction unit vectors of the reflected rays
- diffusion pack: a container with the information on the diffused rays. This will be discussed separately in another tutorial

In [5]:
result = utils_rt.RTXkernel(
    
    
    mesh,              # Mesh object
    ray_origins,       # Coordinates of the origins of the tracing rays
    ray_directions,    # Directions of the tracing rays
    kernel = 'Embree', # Ray tracing kernel (see documentation)
    bounces = 2,       # Number of reflections to compute
    diffusion = False, # Compute the diffused rays after the first reflection?
                                                                            
)

index_tri_container = result[0]
index_ray_container = result[1] 
locations_container = result[2]
ray_origins_container = result[3]
ray_directions_container = result[4]
diffusion_pack = result[5]

Visualize the impinging rays and the hit surfaces.

We first set the colors of the various entities:

In [6]:
base_mesh_color = 'blue'
hit_faces_colors = 'red'
rays_color = 'yellow'

In [7]:
# First Bounce visualization
ray_visualize = tm.load_path(np.hstack(( ray_origins, ray_origins + ray_directions*2.0)).reshape(-1, 2, 3))

mesh.unmerge_vertices()

mesh.visual.face_colors = to_rgba_array(base_mesh_color)#[255, 255, 255, 255]
mesh.visual.face_colors[index_tri_container[0]] = [255,0,0,255]
ray_visualize.colors = np.full((len(ray_origins),4),[238,196,4,100])

scene = tm.Scene([mesh, ray_visualize]) #, ray_visualize2,
scene.show()

IndexError: index 3136 is out of bounds for axis 0 with size 1